# Master merge
The code below is used to create a csv file in which I compile different datasets from different sources.

All individual manipulation and source-aligned merges (merging of different files from same sources) is done exclusively with code, collected in the appropriately named jupyter-notebooks.

## Data Sources
- Substinence data (ADF&G): https://adfg-ak-subsistence.shinyapps.io/CSIS-Data-Downloader/
- Borough and census areas (DCRA): https://dcra-cdo-dcced.opendata.arcgis.com/datasets/d748e3b58c654b64825f974c25b3c697_0/explore
- Unemployment (Laborstats): https://live.laborstats.alaska.gov/labforce/csv/AKlaborforce.csv

# Operations
- Open the dataset in question (souce-specific) as Pandas DataFrame
- Merge into a master DataFrame using namematch to align community or area and year
- Save the end master DataFrame into a csv file for susequent analysis

## Name matching
Since community, borough and region names don't follow the same conventions (some have 'Borough', some have 'Area', etc.) I use this function built on top of the fuzzywuzzy library to check the similarity between strings (names) in a reproduceable and clear way.

The 95-score has been chosen after a bit of trial and error. Manual check of the matchings show no forced ones and very few to none wrongful rejections.

In [41]:
from fuzzywuzzy import process, fuzz

def clean_location_name(name):
    '''
    Cleans location names by removing common suffixes that skew fuzzy matching.
    '''
    ignore_suffixes = ["census designated place", "city", "road", "street", "highway", "borough", "area", "island"]
    for suffix in ignore_suffixes:
        if name.lower().endswith(suffix):
            name = name.rsplit(suffix, 1)[0].strip()
    return name

def match_names(trusted_df, trusted_col, other_df, other_col):
    '''
    Matches names between two dataframe columns, automatically rejecting low-confidence matches.
    
    Returns:
    - Matched DataFrame with original and final names
    - Unmatched names for manual review
    '''

    # Clean and strip names
    trusted_unique = set(trusted_df[trusted_col].astype(str).str.strip().str.lower().apply(clean_location_name).unique())
    other_unique = set(other_df[other_col].astype(str).str.strip().str.lower().apply(clean_location_name).unique())

    matched_other_to_trusted = {}
    unmatched_names = []

    for name in other_unique:
        for trusted_name in trusted_unique:
            if trusted_name in name or name in trusted_name:
                matched_other_to_trusted[name] = trusted_name
                break
        else:
            match, score = process.extractOne(name, list(trusted_unique), scorer=fuzz.token_sort_ratio)
            
            if score >= 95:  # Almost perfect match
                matched_other_to_trusted[name] = match
            else:
                unmatched_names.append(name)  # Log unmatched names

    matched_df = pd.DataFrame(list(matched_other_to_trusted.items()), columns=["Original", "Matched"])
    unmatched_df = pd.DataFrame(unmatched_names, columns=["Unmatched"])

    return matched_df, unmatched_df

# Merging

In [42]:
import os
import pandas as pd

custom_data_directory = '../../data/custom_data'

## Alaskan Department of Fish and Game
I will use this dataset about fishing, hunting and gathering as initial base for the construction of the master dataset to use for statistical analysis.

In [43]:
adfg_df = pd.read_csv(os.path.join(custom_data_directory, 'adfg_custom.csv'))
master_df = adfg_df

## Substinence Use
I add this because it serves as dummy for the recognition of communities as relying on substinence activities for their survival, or not.

In [44]:
substinence_use_df  = pd.read_csv(os.path.join(custom_data_directory, 'substinence_use_status_custom.csv'))
substinence_use_df.columns

Index(['Unnamed: 0', 'commname', 'relying_on_substinence'], dtype='object')

### Name match

In [45]:
matched_df, unmatched_df = match_names(master_df, 'commname', substinence_use_df, 'commname')

### Merge

In [46]:
name_mapping = dict(zip(matched_df["Original"], matched_df["Matched"]))

substinence_use_df["commname"] = substinence_use_df["commname"].replace(name_mapping)

master_df = master_df.merge(substinence_use_df, left_on="commname", right_on="commname", how="left")

master_df.drop(columns=["Unnamed: 0_x", "Unnamed: 0_y"], inplace=True)

## Borough and census area
Since some data is only available per-census area or boroughs - Alaskan equivalent of counties - I will add the census area to each community, from the arcgis database's 'Community and Regions Overview'.

Source: https://dcra-cdo-dcced.opendata.arcgis.com/datasets/d748e3b58c654b64825f974c25b3c697_0/explore

In [47]:
comm_and_reg_df = pd.read_csv(os.path.join(custom_data_directory, 'comm_and_reg_custom.csv'))
comm_and_reg_df.columns

Index(['Unnamed: 0', 'CommunityName', 'IncorporationType', 'BoroughCensusArea',
       'NativeRegionalHealthCarePro', 'SchoolDistrict', 'ClimateRegion',
       'EnergyRegion', 'EconomicRegion', 'sqMiLand', 'sqMiWater'],
      dtype='object')

### Check if all community names are homogeneous

In [48]:
comm_and_reg_comm = set(comm_and_reg_df['CommunityName'].astype(str).str.strip().str.lower().unique().tolist())
master_df_comm = set(master_df['commname'].astype(str).str.strip().str.lower().unique().tolist())

master_df_comm.issubset(comm_and_reg_comm)

False

## Name match and merge

In [49]:
matched_df, unmatched_df = match_names(master_df, 'commname', comm_and_reg_df, 'CommunityName')
# Manually check matches
# matched_df

In [50]:
# Use the matches to create a mapping dictionary
name_mapping = dict(zip(matched_df["Original"], matched_df["Matched"]))

comm_and_reg_df["CommunityName"] = comm_and_reg_df["CommunityName"].replace(name_mapping)

master_df = master_df.merge(comm_and_reg_df, left_on="commname", right_on="CommunityName", how="left")

master_df.drop(columns=["CommunityName"], inplace=True)
master_df.columns

Index(['year', 'commname', 'region', 'subreg', 'resource', 'used', 'trying',
       'hrvsting', 'giving', 'receving', 'xtotnum', 'xtotlbs', 'units',
       'numharv', 'percap', 'totlbhrv', 'mupch', 'pcttotal', 'mupc', 'xupc',
       'commhh', 'relying_on_substinence', 'Unnamed: 0', 'IncorporationType',
       'BoroughCensusArea', 'NativeRegionalHealthCarePro', 'SchoolDistrict',
       'ClimateRegion', 'EnergyRegion', 'EconomicRegion', 'sqMiLand',
       'sqMiWater'],
      dtype='object')

### Rename and reorder columns

In [51]:
merge_col_list = master_df.columns.tolist()
# print(f"{i} \t {x}") for i, x in enumerate(merge_col_list)]
# this order has been manually chosen from the abpove printing of columns
new_merge_col_list = merge_col_list[0:4] + merge_col_list[20:33] + merge_col_list[4:20]

In [52]:
master_df = master_df[new_merge_col_list]
# master_df

In [53]:
master_df.rename(columns={'used': 'using', 
                          'hrvsting': 'harvesting', 
                            'xtotnum': 'units_harvested', 'xtotlbs': 'pounds_harvested', 
                            'units': 'harv_mes_units', 'numharv': 'units_harvested_by_sample', 
                            'percap': 'lbs_harvested_percapita', 'totlbhrv': 'lbs_harvested_by_sample', 
                            'mupch': 'lbs_used_percapita', 'pcttotal': 'perc_contribution_to_harvest',
                            'mupc': 'median_lbs_wildfood_use_percapita',
                            'xupc': 'top_lbs_wildfood_use_percapita', 
                           'commhh': 'adj_households_in_community'}, inplace=True)
master_df.columns

Index(['year', 'commname', 'region', 'subreg', 'adj_households_in_community',
       'relying_on_substinence', 'Unnamed: 0', 'IncorporationType',
       'BoroughCensusArea', 'NativeRegionalHealthCarePro', 'SchoolDistrict',
       'ClimateRegion', 'EnergyRegion', 'EconomicRegion', 'sqMiLand',
       'sqMiWater', 'resource', 'using', 'trying', 'harvesting', 'giving',
       'receving', 'units_harvested', 'pounds_harvested', 'harv_mes_units',
       'units_harvested_by_sample', 'lbs_harvested_percapita',
       'lbs_harvested_by_sample', 'lbs_used_percapita',
       'perc_contribution_to_harvest', 'median_lbs_wildfood_use_percapita',
       'top_lbs_wildfood_use_percapita'],
      dtype='object')

## Merging unemployment data
Since the Unemployment Comparison analysis showed that both the ADFG's and the Alaskan laborstats' dataset contain the same census areas, and the foremost has the most time-varied spread, I will integrate that.

Source: https://live.laborstats.alaska.gov/labforce/csv/AKlaborforce.csv

In [54]:
laborstats_path = '../../data/alaskan_laborstats/'
unem_df = pd.read_csv(os.path.join(laborstats_path, 'Alaska_NOT_SEASONALLY_adjusted.csv'))

### Data prep
The dataset contains monthly data, while only yearly substinence harvest data is available.

I will filter for readily-provided 'Annual' data (identified with period 0, or 'Annual' as Month)

In [55]:
unem_df = unem_df[unem_df['month'] == 'Annual'].copy()

### Dropping columns we don't want and renaming the others for consistency
No need for month, period, or preliminary columns.

In [56]:
cols_to_keep = ['Area Name', 'Year', 'Labor Force', 'Employment', 'Unemployment', 'Unemployment Rate']
unem_df = unem_df[cols_to_keep].copy()

In [57]:
unem_df.rename(columns={'Area Name': 'area_name', 'Year': 'year', 'Labor Force': 'labor_force', 
                        'Employment': 'employed', 'Unemployment': 'unemployed', 
                        'Unemployment Rate': 'unemployment_rate'}, inplace=True)

## Name match

In [58]:
matched_df, unmatched_df = match_names(master_df, 'BoroughCensusArea', unem_df, 'area_name')

### Merge

In [59]:
name_mapping = dict(zip(matched_df["Original"], matched_df["Matched"]))

unem_df["area_name"] = unem_df["area_name"].replace(name_mapping)

master_df = master_df.merge(
    unem_df, 
    left_on=["BoroughCensusArea", "year"],  # Assuming this is the key in master_merge
    right_on=["area_name", "year"], 
    how="left"
)


master_df.drop(columns=["area_name"], inplace=True)

## ACS Files <- maybe arcgis?
Sources for the custom file:
- Household income and benefits: https://dcra-cdo-dcced.opendata.arcgis.com/datasets/5b535254c9f649f9b7ffe52a476052c4_3/explore
- Family Income and Benefits: https://dcra-cdo-dcced.opendata.arcgis.com/datasets/5b535254c9f649f9b7ffe52a476052c4_3/explore?location=50.133297%2C34.600000%2C3.89
- Income and Poverty: https://dcra-cdo-dcced.opendata.arcgis.com/datasets/f4b63e8b2c3042a98130505df1422730_9/explore?location=50.133297%2C34.600000%2C3.89
- Population Characteristics Housing: https://dcra-cdo-dcced.opendata.arcgis.com/datasets/14f0b44db2da4f55b2adf7bb5602bead_6/explore
- Population Characteristics Race: https://dcra-cdo-dcced.opendata.arcgis.com/datasets/30ab75bf594f48349c4ff536957869f8_1/explore?location=42.493141%2C34.600000%2C3.48
- Population Characteristics Sex: https://dcra-cdo-dcced.opendata.arcgis.com/datasets/fa3ad93528b24f7c9ba298a68c04920d_0/explore?location=45.540112%2C34.600000%2C3.63
- Population Characteristics Age: https://dcra-cdo-dcced.opendata.arcgis.com/datasets/63cb9ba34b914f7c89a2394b47d4319e_5/explore?location=46.265046%2C34.600000%2C3.67

In [60]:
arcgis_df = pd.read_csv(os.path.join(custom_data_directory, 'arcgis_custom.csv'))

In [61]:
# arcgis_df

In [62]:
# master_df

In [63]:
# Name match
matched_df, unmatched_df = match_names(master_df, 'BoroughCensusArea', arcgis_df, 'Place')
# matched_df

In [64]:
name_mapping = dict(zip(matched_df['Original'], matched_df['Matched']))

arcgis_df['Place'] = arcgis_df['Place'].replace(name_mapping)

master_df = master_df.merge(arcgis_df, 
                            left_on=['BoroughCensusArea', 'year'], 
                            right_on=['Place', 'Year'], 
                            how='left')

master_df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'Place', 'Year'], inplace=True)

In [65]:
# master_df.columns.tolist()

## Merging IRS data
The original datasets are organized yes by borough and year, but also by tax braket.
What I did in the merging was just to concatenate them to have all years and boroughs, and to compute tax burdens and cleaned up AGI.

I will now aggregate the data to have one row per area (borough) and year, without tax braket division.

source of repository: https://www.irs.gov/downloads/irs-soi?page=17

In [66]:
irs_df = pd.read_csv(os.path.join(custom_data_directory, 'irs_taxes_custom.csv'))
# irs_df.columns.tolist()

In [67]:
import numpy as np

# Aggregate IRS data by year and county
agg_irs = irs_df.groupby(['year', 'county']).apply(
    lambda g: pd.Series({
        'AGI_clean': g['AGI_clean'].sum(),
        'Local_taxes_paid': g['Local_taxes_paid'].sum(),
        'total_tax_burden': (g['AGI_clean'] * g['total_tax_burden']).sum() / g['AGI_clean'].sum() if g['AGI_clean'].sum() > 0 else np.nan,
        'local_tax_burden': (g['AGI_clean'] * g['local_tax_burden']).sum() / g['AGI_clean'].sum() if g['AGI_clean'].sum() > 0 else np.nan,
        'federal_tax_burden': (g['AGI_clean'] * g['federal_tax_burden']).sum() / g['AGI_clean'].sum() if g['AGI_clean'].sum() > 0 else np.nan,
        'Number_of_returns': g['Number_of_returns'].sum(),
        'Gini': (g['Gini'] * g['Number_of_returns']).sum() / g['Number_of_returns'].sum() if g['Number_of_returns'].sum() > 0 else np.nan
    })
).reset_index()

agg_irs['AGI'] = irs_df['Adjusted gross income (AGI)']



/var/folders/0v/j0hxt2dd7zb_l4cbghh38tkc0000gn/T/ipykernel_32709/1547794919.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_irs = irs_df.groupby(['year', 'county']).apply(


In [68]:
# Name match
matched_df, unmatched_df = match_names(master_df, 'BoroughCensusArea', agg_irs, 'county')
name_mapping = dict(zip(matched_df['Original'], matched_df['Matched']))
agg_irs['county'] = agg_irs['county'].replace(name_mapping)

In [69]:
master_df = master_df.merge(agg_irs, left_on=['BoroughCensusArea', 'year'], right_on=['county', 'year'], how='left')
master_df.drop(columns=['county'], inplace=True)

# Computing population density

In [70]:
master_df.columns

Index(['year', 'commname', 'region', 'subreg', 'adj_households_in_community',
       'relying_on_substinence', 'IncorporationType', 'BoroughCensusArea',
       'NativeRegionalHealthCarePro', 'SchoolDistrict', 'ClimateRegion',
       'EnergyRegion', 'EconomicRegion', 'sqMiLand', 'sqMiWater', 'resource',
       'using', 'trying', 'harvesting', 'giving', 'receving',
       'units_harvested', 'pounds_harvested', 'harv_mes_units',
       'units_harvested_by_sample', 'lbs_harvested_percapita',
       'lbs_harvested_by_sample', 'lbs_used_percapita',
       'perc_contribution_to_harvest', 'median_lbs_wildfood_use_percapita',
       'top_lbs_wildfood_use_percapita', 'labor_force', 'employed',
       'unemployed', 'unemployment_rate', 'median_age', 'n_pop_under_14',
       'n_pop_under_18', 'n_pop_18-65', 'n_pop_65-85', 'n_pop_over_85',
       'n_families', 'median_family_income', 'n_households', 'n_houses',
       'n_occupied_houses', 'house_vacancy_rate', 'n_people_below_poverty',
       'n_ho

In [71]:
master_df['pop_density'] = (master_df['n_total_population(ACS)'] / 
                            master_df['sqMiLand'].where(master_df['sqMiLand'] != 0))

In [72]:
master_df['pop_water_density'] = (master_df['n_total_population(ACS)'] / 
                            master_df['sqMiWater'].where(master_df['sqMiLand'] != 0))

# Save as csv

In [73]:
master_df.to_csv(os.path.join(custom_data_directory, 'master.csv'))

In [74]:
master_df.columns.tolist()

['year',
 'commname',
 'region',
 'subreg',
 'adj_households_in_community',
 'relying_on_substinence',
 'IncorporationType',
 'BoroughCensusArea',
 'NativeRegionalHealthCarePro',
 'SchoolDistrict',
 'ClimateRegion',
 'EnergyRegion',
 'EconomicRegion',
 'sqMiLand',
 'sqMiWater',
 'resource',
 'using',
 'trying',
 'harvesting',
 'giving',
 'receving',
 'units_harvested',
 'pounds_harvested',
 'harv_mes_units',
 'units_harvested_by_sample',
 'lbs_harvested_percapita',
 'lbs_harvested_by_sample',
 'lbs_used_percapita',
 'perc_contribution_to_harvest',
 'median_lbs_wildfood_use_percapita',
 'top_lbs_wildfood_use_percapita',
 'labor_force',
 'employed',
 'unemployed',
 'unemployment_rate',
 'median_age',
 'n_pop_under_14',
 'n_pop_under_18',
 'n_pop_18-65',
 'n_pop_65-85',
 'n_pop_over_85',
 'n_families',
 'median_family_income',
 'n_households',
 'n_houses',
 'n_occupied_houses',
 'house_vacancy_rate',
 'n_people_below_poverty',
 'n_households_below_poverty',
 'n_people_under_18_below_pover

## TRYING STUFF

In [174]:
import statsmodels.formula.api as smf

df_to_run = master_df[["median_lbs_wildfood_use_percapita", 
                       "local_tax_burden", 
                       "federal_tax_burden",
                       "unemployment_rate",
                       "AGI", 
                       # "Gini", 
                       "labor_force", 
                       "ratio_male_over_female",
                       "p_population_non_native",
                       "commname", 
                       "year",
                      "resource",
                      "labor_force"]]
df_to_run = df_to_run.dropna()

# Run OLS with community (commname) and year fixed effects
# model = smf.ols(
#     "median_lbs_wildfood_use_percapita ~ local_tax_burden + federal_tax_burden + AGI_clean + n_people_below_poverty + unemployment_rate + ratio_male_over_female + C(commname) + C(year)",
#     data=df_to_run
# ).fit()

model = smf.ols(
    "median_lbs_wildfood_use_percapita ~ local_tax_burden + federal_tax_burden + AGI + labor_force + unemployment_rate + ratio_male_over_female + p_population_non_native + C(commname) + C(year) + C(resource)",
    data=df_to_run
).fit()

print(model.summary())


                                    OLS Regression Results                                   
Dep. Variable:     median_lbs_wildfood_use_percapita   R-squared:                       0.418
Model:                                           OLS   Adj. R-squared:                  0.181
Method:                                Least Squares   F-statistic:                     1.765
Date:                               Thu, 27 Feb 2025   Prob (F-statistic):           0.000912
Time:                                       22:42:53   Log-Likelihood:                -1715.0
No. Observations:                                271   AIC:                             3588.
Df Residuals:                                    192   BIC:                             3872.
Df Model:                                         78                                         
Covariance Type:                           nonrobust                                         
                                             coef    std err

In [175]:
# Overall model summary statistics
print("R-squared:", model.rsquared)
print("Adjusted R-squared:", model.rsquared_adj)
print("F-statistic:", model.fvalue)
print("Overall model p-value:", model.f_pvalue)

# Extract coefficients and p-values for your main regressors
main_vars = ['local_tax_burden', 
             'federal_tax_burden', 
             'AGI', 
             # 'Gini', 
             'unemployment_rate',
             'p_people_below_poverty',
            'ratio_male_over_female',
            'p_population_non_native']
print("\nCoefficients for key variables:")
print(model.params[main_vars])
print("\nP-values for key variables:")
print(model.pvalues[main_vars])


R-squared: 0.41757307384089426
Adjusted R-squared: 0.18096213508875758
F-statistic: 1.764808829393664
Overall model p-value: 0.0009121794943358185

Coefficients for key variables:


KeyError: "['p_people_below_poverty'] not in index"

In [ ]:
import seaborn as sns

sns.lmplot(
    x="p_people_below_poverty",
    y="median_lbs_wildfood_use_percapita",
    hue="commname",  # Different colors for each community
    data=df_to_run,
    height=6,
    aspect=1.5
)

plt.title("Regression: Federal Tax Burden vs. Wild Food Use by Community")
plt.show()


In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

# Ensure population weights are correctly set
weights = master_df.set_index(["BoroughCensusArea", "year"])["n_total_population(ACS)"]

# Aggregate community-level data to the borough level
borough_df = master_df.groupby(["BoroughCensusArea", "year"], observed=True).apply(
    lambda g: pd.Series({
        "median_lbs_wildfood_use_percapita": (
            (g["median_lbs_wildfood_use_percapita"] * g["n_total_population(ACS)"]).sum() / g["n_total_population(ACS)"].sum()
            if g["n_total_population(ACS)"].sum() > 0 else np.nan
        ),
        "units_harvested": g["units_harvested"].sum(),  # Total harvest per borough
        "giving": (
            (g["giving"] * g["n_total_population(ACS)"]).sum() / g["n_total_population(ACS)"].sum()
            if g["n_total_population(ACS)"].sum() > 0 else np.nan
        ),
        "receving": (
            (g["receving"] * g["n_total_population(ACS)"]).sum() / g["n_total_population(ACS)"].sum()
            if g["n_total_population(ACS)"].sum() > 0 else np.nan
        )
    }),
    include_groups=False  # ✅ Fixes the deprecation warning
).reset_index()


# Merge borough-level variables (taxes, unemployment, etc.)
borough_df = borough_df.merge(
    master_df[["BoroughCensusArea", "year", "resource", "local_tax_burden", "federal_tax_burden", 
               "unemployment_rate", "AGI", "labor_force", "ratio_male_over_female", 
               "p_population_non_native"]].drop_duplicates(),
    on=["BoroughCensusArea", "year"],
    how="left"
)

# Filter for years 2012-2022
borough_df = borough_df[(borough_df["year"] >= 2012) & (borough_df["year"] <= 2022)]

# Drop NaN values before regression
borough_df = borough_df.dropna()

# Run the regression at the Borough level
model = smf.ols(
    "median_lbs_wildfood_use_percapita ~ local_tax_burden + federal_tax_burden + AGI + labor_force + unemployment_rate + ratio_male_over_female + p_population_non_native + C(BoroughCensusArea) + C(year) + C(resource)",
    data=borough_df
).fit()

print(model.summary())


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import pandas as pd

# Drop categorical variables before computing VIF
X = borough_df.drop(columns=["median_lbs_wildfood_use_percapita", "year", "BoroughCensusArea", "resource"], errors="ignore")

# Add constant for the intercept
X = sm.add_constant(X)

# Compute VIF
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)


In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Drop highly collinear variable ('giving') to reduce multicollinearity
borough_df = borough_df.drop(columns=["giving"])

# Ensure 'resource' is treated as categorical
borough_df["resource"] = borough_df["resource"].astype("category")

# Define independent and dependent variables
X_cols = ["local_tax_burden", 
          "federal_tax_burden", 
          "AGI", 
          "labor_force", 
          "unemployment_rate", 
          "ratio_male_over_female", 
          "p_population_non_native", 
          "receving",  # Keeping this instead of 'giving'
          "C(resource)", 
          "C(year)", 
          "C(BoroughCensusArea)"]

# Build regression formula
formula = "median_lbs_wildfood_use_percapita ~ " + " + ".join(X_cols)

# Run OLS regression with fixed effects
model = smf.ols(formula=formula, data=borough_df).fit()

# Print regression summary
print(model.summary())

# ---- VIF CHECK ----
# Drop categorical variables before computing VIF
X_vif = borough_df.drop(columns=["median_lbs_wildfood_use_percapita", "BoroughCensusArea", "year", "resource"])
X_vif = sm.add_constant(X_vif)  # Add constant for intercept

# Compute VIF
vif_data = pd.DataFrame()
vif_data["Variable"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# Print VIF results
print("\nVariance Inflation Factor (VIF) Results:")
print(vif_data)


In [ ]:
borough_df["log_wildfood"] = np.log1p(borough_df["median_lbs_wildfood_use_percapita"])


model = smf.ols(
    "log_wildfood ~ federal_tax_burden + AGI + labor_force + unemployment_rate + ratio_male_over_female + p_population_non_native + C(resource) + C(year) + C(BoroughCensusArea)",
    data=borough_df
).fit()

print(model.summary())



In [ ]:
model = smf.ols(formula, data=borough_df).fit(cov_type='HC3')
print(model.summary())


In [ ]:
model = smf.ols(
    "log_wildfood ~ C(BoroughCensusArea) * C(resource) * C(year) + local_tax_burden + federal_tax_burden + AGI + unemployment_rate + ratio_male_over_female + p_population_non_native",
    data=borough_df
).fit()


print(model.summary())
